<h1>Capstone Project - The Battle of the Neighborhoods (Week 2) </h1>

Applied Data Science Capstone by IBM/Coursera <br>

Introduction: Business Problem 
In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an restaurant in Toronto, Canada.<br>

Here we will try finding if someone wants to open a new restaurant in the city which location is best suited for it keeping in mind the competitors and which income group of people will be attracted most to it based on the population of the neighbourhood.<br>

Since there are lots of restaurants in Toronto we will try to detect locations that are not already crowded with restaurants. We would also prefer locations as close to city center as possible, assuming that first two conditions are met.We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.<br>

<b> Data </b>

Based on definition of our problem, factors that will influence our decission are:<br>

All existing restaurants in the neighborhood (any type of restaurant)<br>
Age group of people with their income<br>
Distance of neighborhood from city center<br>
We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.<br>

Following data sources will be needed to extract/generate the required information:<br>

centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M<br>
Number of restaurants and their type and location in every neighborhood will be obtained using Foursquare API

In [1]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)

In [2]:
# define the dataframe columns
column_names = ['Postal_Code','Borough', 'Neighborhood'] 

Nebr = pd.DataFrame(columns=column_names)

In [3]:
from urllib.request import urlopen
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

page = urlopen(wiki)

from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml")

In [4]:
Toronto=soup.find('table', class_='wikitable sortable')

In [5]:
#Generate lists
Pos=[]
Bor=[]
Neig=[]

for row in Toronto.findAll("tr"):
    cells = row.findAll('td')
    if len(cells)==3: 
        Pos.append(cells[0].find(text=True))
        Bor.append(cells[1].find(text=True))
        Neig.append(cells[2].find(text=True))

        
#Add Data to our DataFrame
Nebr['Postal_Code']=Pos
Nebr['Borough']=Bor
Nebr['Neighborhood']=Neig

Nebr.head(20)

,Postal_Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [6]:
Nebr = Nebr.drop(Nebr[Nebr['Borough'].str.contains("Not assigned")==True].index, axis=0, inplace=False)
Nebr.index = pd.RangeIndex(len(Nebr.index))
Nebr.head()

,Postal_Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
Nebr.shape

(103, 3)

In [8]:
column_n = ['NEBRVAL'] 
NEBR_NA = pd.DataFrame(columns=column_n)
NA=['Not assigned'] 
NEBR_NA['NEBRVAL'] = NA

Nebr1=Nebr

for row_index,row in Nebr.iterrows():
    if((Nebr.loc[row_index,['Neighborhood']].values.astype('str') == 'Not assigned') or (Nebr.loc[row_index,['Neighborhood']].values.astype('str') == 'Not assigned\n')):
       
        Nebr1.loc[row_index,['Neighborhood']] = Nebr1.loc[row_index,['Borough']].values.astype('str') 
        
#Reset Index
#nbr.index = pd.RangeIndex(len(nbr.index))

Nebr1.head(20)

,Postal_Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [9]:
Nebr2=Nebr1.groupby('Postal_Code').agg({'Borough':'first',
                               'Neighborhood': ', '.join}).reset_index()

column_names = ['Postal_Code','Borough', 'Neighborhood'] 
Nebr3 = pd.DataFrame(columns=column_names)

Nebr3 = Nebr2.drop(Nebr2[Nebr2['Borough'].str.contains("Toronto")==False].index, axis=0, inplace=False)

#Reset Index
Nebr3.index = pd.RangeIndex(len(Nebr3.index))
Nebr3

,Postal_Code,Borough,Neighborhood
0,M4E,East Toronto,The Beaches\n
1,M4K,East Toronto,"The Danforth West, Riverdale\n"
2,M4L,East Toronto,"India Bazaar, The Beaches West\n"
3,M4M,East Toronto,Studio District\n
4,M4N,Central Toronto,Lawrence Park\n
5,M4P,Central Toronto,Davisville North\n
6,M4R,Central Toronto,"North Toronto West, Lawrence Park\n"
7,M4S,Central Toronto,Davisville\n
8,M4T,Central Toronto,"Moore Park, Summerhill East\n"
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest..."


In [10]:
column = ['Postal_Code','Borough', 'Neighborhood'] 
Nebr_ungrp = pd.DataFrame(columns=column_names)

Nebr_ungrp = Nebr1.drop(Nebr1[Nebr1['Borough'].str.contains("Toronto")==False].index, axis=0, inplace=False)


Nebr_ungrp.index = pd.RangeIndex(len(Nebr_ungrp.index))
Nebr_ungrp

,Postal_Code,Borough,Neighborhood
0,M5A,Downtown Toronto,"Regent Park, Harbourfront"
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
2,M5B,Downtown Toronto,"Garden District, Ryerson"
3,M5C,Downtown Toronto,St. James Town
4,M4E,East Toronto,The Beaches
5,M5E,Downtown Toronto,Berczy Park
6,M5G,Downtown Toronto,Central Bay Street
7,M6G,Downtown Toronto,Christie
8,M5H,Downtown Toronto,"Richmond, Adelaide, King"
9,M6H,West Toronto,"Dufferin, Dovercourt Village"


In [11]:
import time
from geopy.geocoders import Nominatim

In [12]:
from geopy.util import get_version
get_version()

'1.18.1'

In [13]:
geolocator = Nominatim(scheme='http', user_agent="ES1234")

for row_index, item in Nebr_ungrp.iterrows():
    
    list1 = Nebr_ungrp.loc[[row_index],['Neighborhood']].values.astype('str')
    loc = ' , Toronto, Ontario, Canada'
    list1.astype('str')
    list1 = np.append(list1, loc)
    latitude = None
    longitude = None
    location = None
    
    location = geolocator.geocode(list1 , limit = 15)
    time.sleep(5)
    if(location is not None):
        Nebr_ungrp.loc[Nebr_ungrp.index[row_index], 'Latitude'] = location.latitude
        Nebr_ungrp.loc[Nebr_ungrp.index[row_index], 'Longitude'] = location.longitude

In [14]:
print(Nebr_ungrp)

   Postal_Code            Borough                                       Neighborhood   Latitude  Longitude
0         M5A
  Downtown Toronto
                         Regent Park, Harbourfront
        NaN        NaN
1         M7A
  Downtown Toronto
       Queen's Park, Ontario Provincial Government
        NaN        NaN
2         M5B
  Downtown Toronto
                          Garden District, Ryerson
        NaN        NaN
3         M5C
  Downtown Toronto
                                    St. James Town
  43.669403 -79.372704
4         M4E
      East Toronto
                                       The Beaches
  43.671024 -79.296712
5         M5E
  Downtown Toronto
                                       Berczy Park
  43.647984 -79.375396
6         M5G
  Downtown Toronto
                                Central Bay Street
        NaN        NaN
7         M6G
  Downtown Toronto
                                          Christie
  43.664111 -79.418405
8         M5H
  Downtown Toronto
    

In [48]:
import json 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Importing to use the Foursquare API lab

import folium

In [49]:
print('We have {} boroughs and {} neighborhoods.'.format(
        len(Nebr_ungrp['Borough'].unique()),
        Nebr_ungrp.shape[0]
    )
)

Nebr_ungrp.dropna(inplace =True)
Nebr_ungrp.index = pd.RangeIndex(len(Nebr_ungrp.index))

address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="ES1234")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

We have 4 boroughs and 15 neighborhoods.
The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [17]:
Nebr_ungrp

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M5C,Downtown Toronto,St. James Town,43.669403,-79.372704
1,M4E,East Toronto,The Beaches,43.671024,-79.296712
2,M5E,Downtown Toronto,Berczy Park,43.647984,-79.375396
3,M6G,Downtown Toronto,Christie,43.664111,-79.418405
4,M6H,West Toronto,"Dufferin, Dovercourt Village",43.660203,-79.435651
5,M4N,Central Toronto,Lawrence Park,43.729199,-79.403252
6,M5N,Central Toronto,Roselawn,43.710541,-79.401138
7,M4P,Central Toronto,Davisville North,43.697936,-79.397291
8,M6R,West Toronto,"Parkdale, Roncesvalles",43.639875,-79.439653
9,M4S,Central Toronto,Davisville,43.697936,-79.397291


In [18]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(Nebr_ungrp['Latitude'], Nebr_ungrp['Longitude'], Nebr_ungrp['Borough'], Nebr_ungrp['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h1>FOURSQUARE API to explore the neighbourhood </h1>


In [19]:
CLIENT_ID = '0UA4FVK0GXTP5V2R4FIMHUQ0VUQWKJEYTORHIPMCELCCUQV2' # Foursquare ID
CLIENT_SECRET = 'XEPKCCNBHOLECTRZGJX4KGRNRK2RPGTHR34AEM3QHYYJ31PN' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Successfully Logged-In')

Successfully Logged-In


In [20]:
Nebr_ungrp.loc[0]
neighborhood_latitude = np.float(Nebr_ungrp.loc[0,['Latitude']].values)
neighborhood_longitude =  np.float(Nebr_ungrp.loc[0,['Longitude']].values)

<h1>Now, let's get the top 100 venues that are in Harbour Square Park within a radius of 500 meters.</h1>

In [21]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [25]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f398d7418cb1d457213802a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'St. James Town',
  'headerFullLocation': 'St. James Town, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 30,
  'suggestedBounds': {'ne': {'lat': 43.6739032045, 'lng': -79.36649453408027},
   'sw': {'lat': 43.664903195499996, 'lng': -79.37891366591974}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b17c9f5f964a5202fc823e3',
       'name': 'Mr. Jerk',
       'location': {'address': '209 Wellesley St. E',
        'crossStreet': 'btwn Bleecker & Ontario',
        'lat': 43.66732847256732,
        'lng': -79.37338943621165,
        'labeledLatL

In [26]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

<b>Now we are ready to clean the json and structure it into a pandas dataframe.</b>

In [27]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Mr. Jerk,Caribbean Restaurant,43.667328,-79.373389
1,Cranberries,Diner,43.667843,-79.369407
2,Murgatroid,Restaurant,43.667381,-79.369311
3,F'Amelia,Italian Restaurant,43.667536,-79.368613
4,Butter Chicken Factory,Indian Restaurant,43.667072,-79.369184


In [28]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

30 venues were returned by Foursquare.


<h1>2. Explore Neighborhoods in Toronto</h1>

In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
toronto_venues = getNearbyVenues(names=Nebr_ungrp['Neighborhood'],
                                   latitudes=Nebr_ungrp['Latitude'],
                                   longitudes=Nebr_ungrp['Longitude']
                                  )

St. James Town

The Beaches

Berczy Park

Christie

Dufferin, Dovercourt Village

Lawrence Park

Roselawn

Davisville North

Parkdale, Roncesvalles

Davisville

University of Toronto, Harbord

Runnymede, Swansea

Moore Park, Summerhill East

Rosedale

Church and Wellesley



In [31]:
print(toronto_venues.shape)
toronto_venues.head()

(681, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,St. James Town,43.669403,-79.372704,Mr. Jerk,43.667328,-79.373389,Caribbean Restaurant
1,St. James Town,43.669403,-79.372704,Cranberries,43.667843,-79.369407,Diner
2,St. James Town,43.669403,-79.372704,Murgatroid,43.667381,-79.369311,Restaurant
3,St. James Town,43.669403,-79.372704,F'Amelia,43.667536,-79.368613,Italian Restaurant
4,St. James Town,43.669403,-79.372704,Butter Chicken Factory,43.667072,-79.369184,Indian Restaurant


In [32]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,100
Christie,57,57,57,57,57,57
Church and Wellesley,75,75,75,75,75,75
Davisville,40,40,40,40,40,40
Davisville North,40,40,40,40,40,40
"Dufferin, Dovercourt Village",43,43,43,43,43,43
Lawrence Park,50,50,50,50,50,50
"Moore Park, Summerhill East",5,5,5,5,5,5
"Parkdale, Roncesvalles",45,45,45,45,45,45


In [33]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 167 uniques categories.


<h1>3. Analyze Each Neighborhood</h1>

In [34]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Beer Bar,Beer Store,Bike Trail,Bistro,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Line,Café,Campground,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Gastropub,Gay Bar,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym Pool,Health & Beauty Service,Hobby Shop,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Movie Theater,Museum,Music School,Music Store,Nail Salon,New American Restaurant,Noodle House,North Indian Restaurant,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Spa,Sporting Goods Shop,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,St. James Town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,St. James Town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,St. James Town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,St. James Town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,St. James Town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [35]:
toronto_onehot.shape

(681, 168)

In [36]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Beer Bar,Beer Store,Bike Trail,Bistro,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Line,Café,Campground,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Gastropub,Gay Bar,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym Pool,Health & Beauty Service,Hobby Shop,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Movie Theater,Museum,Music School,Music Store,Nail Salon,New American Restaurant,Noodle House,North Indian Restaurant,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Spa,Sporting Goods Shop,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.010000,0.010000,0.02,0.000000,0.000000,0.020000,0.01,0.030000,0.000000,0.000000,0.000000,0.030000,0.000000,0.00,0.010000,0.000000,0.000000,0.020000,0.000000,0.000000,0.00,0.060000,0.00,0.000000,0.000000,0.020000,0.000000,0.000000,0.01,0.000000,0.020000,0.090000,0.000000,0.000000,0.010000,0.010000,0.00,0.010000,0.020000,0.000000,0.010000,0.010000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00000,0.020000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.01,0.01,0.010000,0.000000,0.000000,0.030000,0.00,0.000000,0.01000,0.000000,0.010000,0.030000,0.000000,0.000000,0.000000,0.01,0.040000,0.010000,0.000000,0.000,0.000000,0.000,0.040000,0.040000,0.01,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.010000,0.010000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.01,0.000000,0.020000,0.01,0.000000,0.000000,0.000000,0.00000,0.010000,0.000000,0.000000,0.01,0.00,0.000000,0.000000,0.020000,0.000000,0.000000,0.060000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.03,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000
1,Christie,0.000000,0.000000,0.017544,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.017544,0.000000,0.00,0.035088,0.00,0

In [37]:
toronto_grouped.shape

(15, 168)

<b>Top Venues</b>

In [38]:
Top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(Top_venues))
    print('\n')

----Berczy Park
----
                 venue  freq
0          Coffee Shop  0.09
1           Restaurant  0.06
2                 Café  0.06
3   Italian Restaurant  0.04
4  Japanese Restaurant  0.04


----Christie
----
                venue  freq
0   Korean Restaurant  0.26
1         Coffee Shop  0.05
2      Sandwich Place  0.04
3  Mexican Restaurant  0.04
4       Grocery Store  0.04


----Church and Wellesley
----
                 venue  freq
0          Coffee Shop  0.11
1  Japanese Restaurant  0.07
2     Sushi Restaurant  0.05
3           Restaurant  0.04
4              Gay Bar  0.04


----Davisville
----
                    venue  freq
0      Italian Restaurant  0.10
1        Sushi Restaurant  0.10
2             Coffee Shop  0.08
3       Indian Restaurant  0.05
4  Furniture / Home Store  0.05


----Davisville North
----
                    venue  freq
0      Italian Restaurant  0.10
1        Sushi Restaurant  0.10
2             Coffee Shop  0.08
3       Indian Restaurant  0.05
4  Furnit

In [39]:
def return_most_common_venues(row, Top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:Top_venues]

In [65]:
Top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(Top_venues):
    try:
        columns.append('{}{} Popular Venues'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Popular Venues'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], Top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
0,Berczy Park,Coffee Shop,Restaurant,Café,Italian Restaurant,Japanese Restaurant,Hotel,Gym,Bakery,Beer Bar,Gastropub
1,Christie,Korean Restaurant,Coffee Shop,Grocery Store,Ice Cream Shop,Mexican Restaurant,Cocktail Bar,Sandwich Place,Café,Indian Restaurant,Dessert Shop
2,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Café,Gay Bar,Restaurant,Yoga Studio,Men's Store,Mediterranean Restaurant,Bubble Tea Shop
3,Davisville,Italian Restaurant,Sushi Restaurant,Coffee Shop,Indian Restaurant,Pub,Convenience Store,Pet Store,Furniture / Home Store,Park,Middle Eastern Restaurant
4,Davisville North,Italian Restaurant,Sushi Restaurant,Coffee Shop,Indian Restaurant,Pub,Convenience Store,Pet Store,Furniture / Home Store,Park,Middle Eastern Restaurant
5,"Dufferin, Dovercourt Village",Bar,Bakery,Coffee Shop,Restaurant,Vietnamese Restaurant,Café,Sandwich Place,Mexican Restaurant,Cocktail Bar,Beer Store
6,Lawrence Park,Italian Restaurant,Sushi Restaurant,Bakery,Coffee Shop,Fast Food Restaurant,Pub,Bank,Hobby Shop,Asian Restaurant,Toy / Game Store
7,"Moore Park, Summerhill East",Park,Playground,Candy Store,Grocery Store,Diner,Ethiopian Restaurant,Fish Market,Filipino Restaurant,Department Store,Fast Food Restaurant
8,"Parkdale, Roncesvalles",Tibetan Restaurant,Italian Restaurant,Indian Restaurant,Discount Store,Diner,Pharmacy,Restaurant,Café,Bakery,Pizza Place
9,Rosedale,Playground,Campground,Bike Trail,Park,Ethiopian Restaurant,Flea Market,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market


<h1>4. Cluster Neighborhoodsusing K-Mean</h1>

In [66]:
import folium

In [83]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood',1)
#print(toronto_grouped_clustering)
#print(toronto_grouped)
# run k-means clustering
kmeans = KMeans(init = "k-means++", n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
labels = kmeans.labels_[0:63] 
print(labels)

[0 3 0 4 4 0 4 1 0 2 0 0 0 0 0]


In [80]:
toronto_merged
toronto_merged = Nebr_ungrp
print(toronto_merged.shape)
labels = np.append(labels,labels[0])
print(labels.shape)

(15, 5)
(17,)
